In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.externals import joblib
from sklearn import *
from sklearn import cluster
from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
k_means = joblib.load(open('kmeans.pkl', 'rb'))
loans = pd.read_csv('loanstats.csv')
loans['kcluster'] = k_means.labels_

def manual_clusters(fico):
    if fico > 720:
        return 'Group1'
    elif fico > 700:
        return 'Group2'
    elif fico > 690: 
        return 'Group3'
    elif fico > 680: # amnt < 25000 
        return 'Group4'
    elif fico > 670:
        return 'Group5'
    else:
        return 'Group6'

loans['manual_cat'] = loans.apply(lambda x: manual_clusters(x['fico_range_low']), axis=1)

loans['term'] = loans['term'].map(lambda a: int(a.strip(' months')))
loans['application_type'] = loans['application_type'].map(lambda a: 1 if a=='Joint App' else 0)

def elength(a):
    if (a=='n/a'):
        return 0
    elif (a=='10+ years'):
        return 10
    elif (a=='1 year'):
        return 1
    elif (a=='< 1 year'):
        return 0.5
    else:
        return float(a.strip(' years'))
    
loans['emp_length'] = loans['emp_length'].map(lambda a: elength(a))

loans['revol_util'] = loans['revol_util'].map(lambda a: float(a.strip('%')))

homes = pd.get_dummies(loans['home_ownership'], prefix='home')
loans = loans.join(homes)
loans.drop('home_ownership', axis=1, inplace=True)

states = pd.get_dummies(loans['addr_state'], prefix='st')
loans = loans.join(states)
loans.drop('addr_state', axis=1, inplace=True)

loans.drop('grade', axis=1, inplace=True)
loans.drop('sub_grade', axis=1, inplace=True)
loans.drop('set', axis=1, inplace=True)
loans.drop('emp_title', axis=1, inplace=True)
loans.drop('timestamp', axis=1, inplace=True)
loans.drop('issue_d', axis=1, inplace=True)
loans.drop('last_credit_pull_d', axis=1, inplace=True)
loans.drop('title', axis=1, inplace=True)
loans.drop('purpose', axis=1, inplace=True)
loans.drop('next_pymnt_d', axis=1, inplace=True)
loans.drop('zip_code', axis=1, inplace=True)
loans.drop('last_fico_range_high', axis=1, inplace=True)
loans.drop('fico_range_high', axis=1, inplace=True)
loans.drop('last_fico_range_low', axis=1, inplace=True)
loans.drop('installment', axis=1, inplace=True)

In [3]:
man1 = loans[loans['manual_cat']=='Group1']
man2 = loans[loans['manual_cat']=='Group2']
man3 = loans[loans['manual_cat']=='Group3']
man4 = loans[loans['manual_cat']=='Group4']
man5 = loans[loans['manual_cat']=='Group5']
man6 = loans[loans['manual_cat']=='Group6']

man1.drop('manual_cat',axis=1,inplace=True)
man2.drop('manual_cat',axis=1,inplace=True)
man3.drop('manual_cat',axis=1,inplace=True)
man4.drop('manual_cat',axis=1,inplace=True)
man5.drop('manual_cat',axis=1,inplace=True)
man6.drop('manual_cat',axis=1,inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11

In [4]:
array=man1.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [5]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(n_estimators=60)

In [6]:
model.fit(X_train,Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [7]:
ptrain = model.predict(X_train)
ptest = model.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |1.036432526729083|2.760097762097486|
|mae    |0.742317542859    |1.98803859024    |
|mape   |7.70770673283    |20.5805494099    |


In [8]:
array=man2.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
model=RandomForestRegressor(n_estimators=60)
model.fit(X_train,Y_train)
ptrain = model.predict(X_train)
ptest = model.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |1.1921831525362583|3.1481481917251553|
|mae    |0.888603041371    |2.36979415301    |
|mape   |7.90711483796    |21.0568556924    |


In [9]:
array=man3.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
model=RandomForestRegressor(n_estimators=60)
model.fit(X_train,Y_train)
ptrain = model.predict(X_train)
ptest = model.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |1.2562807320581457|3.304693519817137|
|mae    |0.940833172761    |2.49792174929    |
|mape   |7.73893038609    |20.541319919    |


In [10]:
array=man4.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
model=RandomForestRegressor(n_estimators=60)
model.fit(X_train,Y_train)
ptrain = model.predict(X_train)
ptest = model.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |1.286606665888402|3.3967386977001204|
|mae    |0.967681864163    |2.58808358111    |
|mape   |7.58259091007    |20.4132749314    |


In [11]:
array=man5.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
model=RandomForestRegressor(n_estimators=60)
model.fit(X_train,Y_train)
ptrain = model.predict(X_train)
ptest = model.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |1.2813483904846819|3.380414977089708|
|mae    |0.964418088938    |2.57498048159    |
|mape   |7.13744976184    |19.0032752238    |


In [12]:
array=man6.values
Y=array[:,3]
x1=array[:,1:3]
x2=array[:,4:]
X=np.hstack((x1,x2))
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
model=RandomForestRegressor(n_estimators=60)
model.fit(X_train,Y_train)
ptrain = model.predict(X_train)
ptest = model.predict(X_test)
rms_train = sqrt(mean_squared_error(Y_train, ptrain))
rms_test = sqrt(mean_squared_error(Y_test, ptest))
mae_train = mean_absolute_error(Y_train, ptrain)
mae_test = mean_absolute_error(Y_test, ptest)
mape_train = np.mean(np.abs((Y_train - ptrain) / Y_train)) * 100
mape_test = np.mean(np.abs((Y_test - ptest) / Y_test)) * 100

print('|metric |train            |test             | \n|rms    |'+str(rms_train)+'|'+str(rms_test)+'|\n|mae    |'
      +str(mae_train)+'    |'+str(mae_test)+'    |\n|mape   |'+str(mape_train)+'    |'+str(mape_test)+'    |')

|metric |train            |test             | 
|rms    |1.2655297823201705|3.346537927180759|
|mae    |0.956898532709    |2.56669740408    |
|mape   |6.63770216348    |17.8059996841    |
